### Bước 1: Cài đặt các thư viện cần thiết

In [1]:
!pip install tensorflow
!pip install split-folders
!pip install albumentations

import os
import tensorflow as tf
import random
import numpy as np
import splitfolders
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
import matplotlib.pyplot as plt
from PIL import Image

### Bước 2: Đặt đường dẫn home chứa folder project hiện tại

In [2]:
import platform

if platform.system() == 'Linux':
    home_dir = '/content/drive/MyDrive/ClassificationofMangoDiseases'
elif platform.system() == 'Windows':
    home_dir = 'D:\Projects\ClassificationofMangoDiseases'
else:
    raise ValueError('Unsupported platform')

print(f'Home directory: {home_dir}')

### Bước 3: Kết nối với Google Drive nếu đang chạy trên Google Colab

In [3]:
if platform.system() == 'Linux':
    from google.colab import drive
    drive.mount('/content/drive')

### Bước 4: Đặt đường dẫn tới tập dữ liệu

In [4]:
# Đường dẫn tới tập dữ liệu đã chọn (gốc hoặc đã remove background)
data_dir = os.path.join(home_dir, 'data/MangoFruitDDS/SenMangoFruitDDS_bgremoved')  # Hoặc 'SenMangoFruitDDS_original'
output_dir = os.path.join(home_dir, 'data/processed')
augmented_dir = os.path.join(home_dir, 'data/augmented')

print(f'Data directory: {data_dir}')
print(f'Output directory: {output_dir}')
print(f'Augmented directory: {augmented_dir}')

### Bước 5: Tiền xử lý dữ liệu và phân chia tập dữ liệu

Sử dụng thư viện `split-folders` để phân chia tập dữ liệu thành các tập `train`, `valid`, và `test`.

In [5]:
# Xóa thư mục đầu ra nếu đã tồn tại
if os.path.exists(output_dir):
    import shutil
    shutil.rmtree(output_dir)

# Phân chia dữ liệu
splitfolders.ratio(data_dir, output=output_dir, seed=42, ratio=(.7, .2, .1), group_prefix=None)

### Bước 6: Tăng cường dữ liệu

In [6]:
# Định nghĩa các phép tăng cường dữ liệu
augmentations = A.Compose([
    A.RandomRotate90(),
    A.Flip(),
    A.Transpose(),
    A.OneOf([
        A.GaussNoise(),
        A.MultiplicativeNoise()
    ], p=0.2),
    A.OneOf([
        A.MotionBlur(p=0.2),
        A.MedianBlur(blur_limit=3, p=0.1),
        A.Blur(blur_limit=3, p=0.1),
    ], p=0.2),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=20, p=0.2),
    A.OneOf([
        A.OpticalDistortion(p=0.3),
        A.GridDistortion(p=0.1),
        A.IAAPiecewiseAffine(p=0.3),
    ], p=0.2),
    A.OneOf([
        A.CLAHE(clip_limit=2),
        A.IAASharpen(),
        A.IAAEmboss(),
        A.RandomBrightnessContrast(),
    ], p=0.3),
    A.HueSaturationValue(p=0.3),
])

# Tạo thư mục đầu ra cho dữ liệu tăng cường
if not os.path.exists(augmented_dir):
    os.makedirs(augmented_dir)

# Áp dụng tăng cường dữ liệu
for root, dirs, files in os.walk(output_dir):
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(root, file)
            img = np.array(Image.open(img_path))
            augmented_img = augmentations(image=img)['image']
            augmented_img = Image.fromarray(augmented_img)
            output_path = img_path.replace(output_dir, augmented_dir)
            os.makedirs(os.path.dirname(output_path), exist_ok=True)
            augmented_img.save(output_path)

### Bước 7: Hiển thị thông tin thư mục sau khi phân chia và tăng cường

In [7]:
# Kiểm tra các thư mục sau khi phân chia
for root, dirs, files in os.walk(output_dir):
    level = root.replace(output_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print('{}{}/'.format(indent, os.path.basename(root)))
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print('{}{}'.format(subindent, f))

### Bước 8: Định nghĩa các hàm tạo mô hình

In [8]:
# Set random seed for reproducibility
random_seed = 42
np.random.seed(random_seed)
tf.random.set_seed(random_seed)
random.seed(random_seed)

# Define common hyperparameters
batch_size = 32
epochs = 100
learning_rate = 0.001

# Define common loss function and optimizer
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Define image size and input shape
img_height = 224
img_width = 224
input_shape = (img_height, img_width, 3)

# Function to create a simple CNN model
def create_cnn_model(input_shape, num_classes):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a ResNet model
def create_resnet_model(input_shape, num_classes):
    base_model = tf.keras.applications.ResNet50(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create an Inception model
def create_inception_model(input_shape, num_classes):
    base_model = tf.keras.applications.InceptionV3(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a DenseNet model
def create_densenet_model(input_shape, num_classes):
    base_model = tf.keras.applications.DenseNet121(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Function to create a model using Transfer Learning
def create_transfer_learning_model(input_shape, num_classes):
    base_model = tf.keras.applications.VGG16(
        input_shape=input_shape,
        include_top=False,
        weights='imagenet'
    )
    base_model.trainable = False

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model

### Bước 9: Tạo các DataLoader và huấn luyện mô hình

In [9]:
# Định nghĩa hàm huấn luyện và lưu mô hình
def train_and_save_model(model, train_data, val_data, model_name, save_dir):
    history = model.fit(
        train_data,
        epochs=epochs,
        validation_data=val_data
    )
    model.save(os.path.join(save_dir, model_name + '.h5'))
    return history

# Thiết lập đường dẫn lưu mô hình
if platform.system() == 'Linux':
    model_save_dir = '/content/drive/MyDrive/ClassificationofMangoDiseases/models'
elif platform.system() == 'Windows':
    model_save_dir = 'D:\Projects\ClassificationofMangoDiseases\models'
else:
    raise ValueError('Unsupported platform')

# Đảm bảo thư mục lưu trữ tồn tại
os.makedirs(model_save_dir, exist_ok=True)

# Tạo data generators
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    os.path.join(output_dir, 'train'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

val_data = val_datagen.flow_from_directory(
    os.path.join(output_dir, 'val'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

aug_train_data = train_datagen.flow_from_directory(
    os.path.join(augmented_dir, 'train'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

aug_val_data = val_datagen.flow_from_directory(
    os.path.join(augmented_dir, 'val'),
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse'
)

# Define the number of classes (adjust this according to your dataset)
num_classes = len(train_data.class_indices)

# Tạo và huấn luyện các mô hình
model_names = ['CNNCustomOri', 'ResNet50Ori', 'InceptionV3Ori', 'DenseNet121Ori', 'VGG16Ori', 
               'CNNCustomAug', 'ResNet50Aug', 'InceptionV3Aug', 'DenseNet121Aug', 'VGG16Aug']

# Original data models
cnn_model = create_cnn_model(input_shape, num_classes)
resnet_model = create_resnet_model(input_shape, num_classes)
inception_model = create_inception_model(input_shape, num_classes)
densenet_model = create_densenet_model(input_shape, num_classes)
vgg16_model = create_transfer_learning_model(input_shape, num_classes)

models_ori = [cnn_model, resnet_model, inception_model, densenet_model, vgg16_model]

# Augmented data models
cnn_model_aug = create_cnn_model(input_shape, num_classes)
resnet_model_aug = create_resnet_model(input_shape, num_classes)
inception_model_aug = create_inception_model(input_shape, num_classes)
densenet_model_aug = create_densenet_model(input_shape, num_classes)
vgg16_model_aug = create_transfer_learning_model(input_shape, num_classes)

models_aug = [cnn_model_aug, resnet_model_aug, inception_model_aug, densenet_model_aug, vgg16_model_aug]

# Compile the models
for model in models_ori + models_aug:
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

# Train models on original data
for model, name in zip(models_ori, model_names[:5]):
    train_and_save_model(model, train_data, val_data, name, model_save_dir)

# Train models on augmented data
for model, name in zip(models_aug, model_names[5:]):
    train_and_save_model(model, aug_train_data, aug_val_data, name, model_save_dir)